In [1]:
import cv2
import numpy as np

# Load the image in grayscale
image = cv2.imread('lib3.png', cv2.IMREAD_GRAYSCALE)

threshold = 240  # Adjust this value as needed

# Create a 2D matrix of the same size as the image
height, width = image.shape
result_matrix = np.zeros((height, width), dtype=np.int32)

# Fill the cells based on the threshold
for i in range(height):
    for j in range(width):
        if image[i, j] < threshold:
            result_matrix[i, j] = 0
        else:
            result_matrix[i, j] = -1

#draw_matrix(result_matrix)
print("done loading image")

done loading image


In [2]:
import queue

def mark_pixel(x, y, value, pixel_queue):
    if x < 0 or y < 0 or x >= height or y >= width:
        return
    
    if result_matrix[x, y] != -1:
        return
    
    result_matrix[x, y] = value
    pixel_queue.put([x,y])

# Create a FIFO queue
pixel_queue = queue.Queue()

# enqueue all walls (v==0)

for i in range(height):
    for j in range(width):
        if result_matrix[i, j] == 0:
            pixel_queue.put([i, j])
            

# process each pixel
while not pixel_queue.empty():
    # print(pixel_queue.qsize(), " || ", i, "-", j)
    [i, j] = pixel_queue.get()
    current_value = result_matrix[i, j] + 1
    mark_pixel(i-1, j, current_value, pixel_queue)
    mark_pixel(i+1, j, current_value, pixel_queue)
    mark_pixel(i, j-1, current_value, pixel_queue)
    mark_pixel(i, j+1, current_value, pixel_queue)
    

print("done lableing distance to wall")

done lableing distance to wall


In [3]:
def create_2_matrix(result_matrix):
    new_matrix = np.empty_like(result_matrix, dtype=np.str_)
    door_matrix = result_matrix.copy()
    height, width = image.shape
    for i in range(height):
        for j in range(width):
            current_value = result_matrix[i, j]

            # Get the values of adjacent cells
            up_value = result_matrix[i - 1, j] if i > 0 else None
            down_value = result_matrix[i + 1, j] if i < height - 1 else None
            left_value = result_matrix[i, j - 1] if j > 0 else None
            right_value = result_matrix[i, j + 1] if j < width - 1 else None

            if current_value == 0:
                new_matrix[i, j] = '0'
            elif (up_value is None or current_value >= up_value) and \
                 (down_value is None or current_value >= down_value) and \
                 (left_value is None or current_value >= left_value) and \
                 (right_value is None or current_value >= right_value):
                new_matrix[i, j] = '*'
            else:
                new_matrix[i, j] = '.'
    get_map_doors(height, width, new_matrix, door_matrix)
    return new_matrix, door_matrix
    
def change_door_x(path_clear, path, new_matrix, door_matrix):
    if path_clear:
        for x, y in path:
            new_matrix[x, y] = 'X'
            door_matrix[x, y] = 0
            

def get_map_doors(height, width, new_matrix, door_matrix):
    for i in range(height):
        for j in range(width):
            if new_matrix[i, j] == '0':
                path = []

                # Check up
                x = i - 1
                while x >= 0 and new_matrix[x, j] != '0':
                    if new_matrix[x, j] == '.':
                        path.append((x, j))
                    else:
                        path = []
                        path_clear = False
                        break
                    x -= 1
                    if x < 0:
                        path = []
                        path_clear = False
                        break
                    if len(path) > 0:
                        path_clear = True
                
                change_door_x(path_clear, path, new_matrix, door_matrix)

                # Check down
                x = i + 1
                while x < height and new_matrix[x, j] != '0':
                    if new_matrix[x, j] == '.':
                        path.append((x, j))
                    else:
                        path = []
                        path_clear = False
                        break
                    x += 1
                    if x >= height:
                        path = []
                        path_clear = False
                        break
                    if len(path) > 0:
                        path_clear = True

                change_door_x(path_clear, path, new_matrix, door_matrix)

                # Check left
                y = j - 1
                while y >= 0 and new_matrix[i, y] != '0':
                    if new_matrix[i, y] == '.':
                        path.append((i, y))
                    else:
                        path = []
                        path_clear = False
                        break
                    y -= 1
                    if y < 0:
                        path = []
                        path_clear = False
                        break
                    if len(path) > 0:
                        path_clear = True

                change_door_x(path_clear, path, new_matrix, door_matrix)

                # Check right
                y = j + 1
                while y < width and new_matrix[i, y] != '0':
                    if new_matrix[i, y] == '.':
                        path.append((i, y))
                    else:
                        path = []
                        path_clear = False
                        break
                    y += 1
                    if y >= width:
                        path = []
                        path_clear = False
                        break
                    if len(path) > 0:
                        path_clear = True

                change_door_x(path_clear, path, new_matrix, door_matrix)
                
door_matrix, new_matrix = create_2_matrix(result_matrix)
print("done door identification and room seperation")

done door identification and room seperation


In [4]:
#print_matrix(result_matrix)
#print("================")
#print_matrix(new_matrix)

In [5]:
def draw_matrix(image):
    new_img = np.array(image.copy())
    # Convert -1 values to 255 (white) for visualization
    new_img[new_img == -1] = 127
    
    new_img = new_img.astype(np.uint8)

    print(new_img.shape)
    # Combine the mask with the original image to draw the cells
    cv2.imshow("Grayscale Image", new_img)
    cv2.waitKey(0)  # This line allows you to close the image window by pressing any key
    cv2.destroyAllWindows()
    
def print_matrix(matrix):
    height, width = image.shape
    for i in range(height):
        for j in range(width):
            print(matrix[i, j], end=' ')
        print()  # Move to the next row

In [6]:
import random
import time


pivot_count = 0
def random_color():
    red = np.int32(random.randint(150, 255))
    green = np.int32(random.randint(150, 255))
    blue = np.int32(random.randint(150, 255))
    return (red, green, blue)

def is_vaild_index(index, verify_painted_matrix):
    [x, y] = index
    num_rows = len(verify_painted_matrix)
    num_cols = len(verify_painted_matrix[0])
    
    if not (0 <= x < num_rows and 0 <= y < num_cols):
        return False
    
    if verify_painted_matrix[x][y] == True:
        return False
    
    # in range, not painted
    return True
    
def is_not_all_painted(verify_painted_matrix):
    for row in verify_painted_matrix:
        for value in row:
            if value == False:
                return True
            
    return False

def find_max(distance_matrix, verify_painted_matrix):
    max_val = 0
    cor = [-1, -1]
    
    for i in range(len(distance_matrix)):
        for j in range(len(distance_matrix[0])):
            if verify_painted_matrix[i][j] == False:
                # not paint
                if distance_matrix[i][j] > max_val:
                    max_val = distance_matrix[i][j]
                    cor = [i, j]
    
    return cor

def paint_room(cordinate, distance_matrix, verify_painted_matrix, colored_image, count):
    print("paint room of: ",cordinate)
    [x, y] = cordinate
    
    color = random_color()
    
    room_pixel_queue = queue.Queue()
    room_pixel_queue.put(cordinate)
    
    r_val = 50 + count * 20
    if r_val > 255:
        r_val = 255
    colored_image[x][y] = (np.int32(0), np.int32(0), np.int32(r_val))
    verify_painted_matrix[x][y] = True
    count = 0
    while not room_pixel_queue.empty():
        cor = room_pixel_queue.get()
        # print(cor)
        x = cor[0]
        y = cor[1]
        curent_value = distance_matrix[x][y]
        count = count + 1
        
        x_n = x -1
        y_n = y
        if is_vaild_index([x_n, y_n], verify_painted_matrix) and distance_matrix[x_n][y_n] != 0:
            # able to paint
            colored_image[x_n][y_n] = color
            verify_painted_matrix[x_n][y_n] = True
            room_pixel_queue.put([x_n, y_n])
                
        x_n = x + 1
        y_n = y
        if is_vaild_index([x_n, y_n], verify_painted_matrix) and distance_matrix[x_n][y_n] != 0:
            # able to paint
            colored_image[x_n][y_n] = color
            verify_painted_matrix[x_n][y_n] = True
            room_pixel_queue.put([x_n, y_n])
                
        x_n = x
        y_n = y - 1
        if is_vaild_index([x_n, y_n], verify_painted_matrix) and distance_matrix[x_n][y_n] != 0:
            # able to paint
            colored_image[x_n][y_n] = color
            verify_painted_matrix[x_n][y_n] = True
            room_pixel_queue.put([x_n, y_n])
                
        x_n = x
        y_n = y + 1
        if is_vaild_index([x_n, y_n], verify_painted_matrix) and distance_matrix[x_n][y_n] != 0:
            # able to paint
            colored_image[x_n][y_n] = color
            verify_painted_matrix[x_n][y_n] = True
            room_pixel_queue.put([x_n, y_n])
    
    print(count)
        

def create_colored_room(distance_matrix, colored_image):
    height = len(distance_matrix)
    width = len(distance_matrix[0])
    
    verify_painted_matrix = [[False for _ in range(width)] for _ in range(height)]
                
    count = 0
    while is_not_all_painted(verify_painted_matrix):
        # find max
        cordinate = find_max(distance_matrix, verify_painted_matrix)
        if cordinate == [-1, -1]:
            break
            
        paint_room(cordinate, distance_matrix, verify_painted_matrix, colored_image, count)
        count = count + 1
        
    
    print("all room done")
    return colored_image
    
    # locate the higest value

In [7]:
pivot_count = 0
colored_image = np.zeros((height, width, 3), dtype=np.uint8)
create_colored_room(new_matrix, colored_image)

print("das")
print(colored_image.shape)
print(colored_image[0][0])

paint room of:  [1960, 0]
743307
paint room of:  [736, 731]
229346
paint room of:  [1429, 1188]
174142
paint room of:  [382, 646]
436705
paint room of:  [311, 250]
125689
paint room of:  [755, 350]
171068
paint room of:  [1596, 745]
101120
paint room of:  [1607, 230]
76646
paint room of:  [186, 1177]
37506
paint room of:  [511, 1176]
51185
paint room of:  [527, 33]
46847
paint room of:  [1293, 728]
14981
paint room of:  [1095, 1076]
32488
paint room of:  [1212, 983]
17976
paint room of:  [1713, 710]
7790
paint room of:  [1215, 1037]
12760
paint room of:  [1084, 168]
59070
paint room of:  [125, 1040]
10074
paint room of:  [196, 962]
9946
paint room of:  [503, 314]
5544
paint room of:  [1187, 180]
6803
paint room of:  [585, 252]
6750
paint room of:  [1212, 1014]
366
paint room of:  [1400, 681]
4391
paint room of:  [1400, 803]
4233
paint room of:  [500, 157]
3998
paint room of:  [1443, 316]
4801
paint room of:  [1452, 387]
7276
paint room of:  [1518, 982]
4104
paint room of:  [849, 1070]


In [8]:
def mark_path(result_matrix, colored_image):
    height = len(result_matrix)
    width = len(result_matrix[0])
    
    for i in range(height):
        for j in range(width):
            current_value = result_matrix[i, j]

            # Get the values of adjacent cells
            up_value = result_matrix[i - 1, j] if i > 0 else None
            down_value = result_matrix[i + 1, j] if i < height - 1 else None
            left_value = result_matrix[i, j - 1] if j > 0 else None
            right_value = result_matrix[i, j + 1] if j < width - 1 else None

            if (up_value is None or current_value >= up_value) and \
               (down_value is None or current_value >= down_value) and \
               (left_value is None or current_value >= left_value) and \
               (right_value is None or current_value >= right_value):
                colored_image[i][j] = (np.int32(0), np.int32(0), np.int32(0))

In [9]:

#mark_path(result_matrix, colored_image)
cv2.imwrite("lib_34.png", colored_image)

True

In [10]:
#print_matrix(new_matrix)

In [11]:
# get path way
def local_highest(door_matrix, new_matrix):
    entry_matrix = new_matrix.copy()
    height, width = door_matrix.shape
    for i in range(height):
        for j in range(width):
            if door_matrix[i, j] == '*':
                entry_matrix[i, j] = 1
            else:
                entry_matrix[i, j] = 0
    
    return entry_matrix

In [12]:
path_matrix = local_highest(door_matrix, new_matrix)

def mark_path(path_matrix, colored_image):
    path_image = colored_image.copy()
    height, width = path_matrix.shape
    for i in range(height):
        for j in range(width):
            if path_matrix[i][j] == 1:
                path_image[i][j] = (np.int32(240), np.int32(0), np.int32(0))
            
    return path_image


path_image = mark_path(path_matrix, colored_image)

cv2.imwrite("lib_path_1.png", path_image)

True